# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 

## Import Packages

In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from numpy.polynomial.polynomial import polyfit
import cv2
%matplotlib inline

## Helper Functions

In [3]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines_old(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    # right and left lane slope average
    right_slope_sum  = 0
    left_slope_sum = 0
    # right and left position average through which the red lines should pass 
    right_x_pos_sum = 0
    right_y_pos_sum = 0
    left_x_pos_sum = 0
    left_y_pos_sum = 0
    # number of segments in the right and left lane
    num_seg_right = 0
    num_seg_left = 0
    for line in lines:
        for x1,y1,x2,y2 in line:
            # calulate slope of line
            slope = (y1 - y2) / (x1 - x2) 
            # if the segments is a part of the right lane (negative slope)
            if (slope < 0):
                # increment number of segments in the right slope
                num_seg_right = num_seg_right + 1 
                # add to slope sum
                right_slope_sum = right_slope_sum + slope
                # add the position to the position sum
                right_x_pos_sum = right_x_pos_sum + (x1 + x2) / 2
                right_y_pos_sum = right_y_pos_sum + (y1 + y2) / 2
            elif (slope > 0):  # don't consider horizontal lines
                # increment number of segments in the right slope
                num_seg_left = num_seg_left + 1 
                # add to slope sum
                left_slope_sum = left_slope_sum + slope
                # add the position to the position sum
                left_x_pos_sum = left_x_pos_sum + (x1 + x2) / 2
                left_y_pos_sum = left_y_pos_sum + (y1 + y2) / 2
    # calculate average slope for the right an left lane
    right_slope_av = right_slope_sum / num_seg_right
    left_slope_av = left_slope_sum / num_seg_left
    # calculate postion average
    right_x_pos_av = right_x_pos_sum / num_seg_right
    right_y_pos_av = right_y_pos_sum / num_seg_right
    left_x_pos_av = left_x_pos_sum / num_seg_left
    left_y_pos_av = left_y_pos_sum / num_seg_left
    # determine b in y = a . x + b
    b_right = right_y_pos_av - right_slope_av * right_x_pos_av
    b_left = left_y_pos_av - left_slope_av * left_x_pos_av
    # determine the start and end point of the segment
    start_left_y = img.shape[0]
    start_left_x = (start_left_y - b_left) / left_slope_av 
    end_left_y = img.shape[0] * 0.6 
    end_left_x = (end_left_y - b_left) / left_slope_av 
    
    start_right_y = img.shape[0]
    start_right_x = (start_right_y - b_right) / right_slope_av 
    end_right_y = img.shape[0] * 0.6 
    end_right_x = (end_right_y - b_right) / right_slope_av   
    # draw right lane
    cv2.line(img, (int(round(start_right_x)), int(round(start_right_y))), 
                   (int(round(end_right_x)), int(round(end_right_y))), color, thickness)
    # draw left lane
    cv2.line(img, (int(round(start_left_x)), int(round(start_left_y))), 
                   (int(round(end_left_x)), int(round(end_left_y))), color, thickness)

def draw_lines (img, lines, color=[255, 0, 0], thickness=5):
    # initiate point list for right and left lane
    left_lane_x = []
    left_lane_y = []
    right_lane_x = []
    right_lane_y = []
    # initiate slope
    slope = 1
    # iterate through lines
    for line in lines:
        # for points in line
        for x1,y1,x2,y2 in line:
            # calculate slope
            slope = (y1 - y2) / (x1 - x2)
            # add points to corresponding lane
            if (slope > 0): #and (((x1+x2)/2) < (img.shape[1] / 2)) : # if slope is positive to the left
                left_lane_x.extend([x1, x2])
                left_lane_y.extend([y1, y2])
            elif (slope < 0):# and (((x1+x2)/2) > (img.shape[1] / 2)):  # if slope is negative to the right
                right_lane_x.extend([x1, x2])
                right_lane_y.extend([y1, y2])
    
    # if at a given time no segments are detected for a given lanes
    #if (not right_lane_x) or (not left_lane_y):  # don't draw a line
     #   return
    if  (right_lane_x) and (left_lane_y): # if there is something to draw
        # determine equation desscribing the lane line
        b_right, m_right = polyfit(np.array(right_lane_x), np.array(right_lane_y), 1)
        b_left, m_left = polyfit(np.array(left_lane_x), np.array(left_lane_y), 1)
        
        # determine x-position of start point given y = shape[0]
        start_right_y = img.shape[0]  # max(right_lane_y)  
        start_right_x = int(round((start_right_y - b_right) / 
                                  m_right)) if m_right != 0 else int(round(img.shape[1]*0.9)) 
        start_left_y = img.shape[0] # max(left_lane_y) 
        start_left_x = int(round((start_left_y - b_left) / 
                                  m_left)) if m_left != 0 else int(round(img.shape[1]*0.1))
        
        # ending point 
        end_right_y =  max(int(round(img.shape[0] * 0.6)), min(right_lane_y)) # min(right_lane_y)  
        end_right_x = int(round((end_right_y - b_right) / 
                                  m_right)) if m_right != 0 else int(round(img.shape[0]*0.9)) 
        end_left_y =  max(int(round(img.shape[0] * 0.6)), min(right_lane_y))  # min(right_lane_y)  
        end_left_x = int(round((end_left_y - b_left) / 
                                  m_left)) if m_left != 0 else int(round(img.shape[0]*0.1))
        
        # draw lines
        cv2.line(img, (start_left_x, start_left_y), 
                   (end_left_x, end_left_y), color, thickness)
        cv2.line(img, (start_right_x, start_right_y), 
                   (end_right_x, end_right_y), color, thickness)
    
    return
    
      
def draw_line_segments(img, lines, color=[255, 0, 0], thickness=5):
    """
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
        

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.7, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [4]:
import os
images_list = os.listdir("test_images/")  # show the list of images in test_images/

## Build a Lane Finding Pipeline


Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

# TODO: Build your pipeline that will draw lane lines on the test_images
i = 0 
for image_name in images_list:
    # read and grayscale the image
    image = mpimg.imread('test_images/' + image_name)
    imshape = image.shape  # get the image shape
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # apply Gaussian smoothing
    blur_gray = gaussian_blur(gray, kernel_size=5)
    # apply Canny Edge Detection
    edges = canny(blur_gray, low_threshold=50, high_threshold=150)
    # mask the region we are not intersted in
    vertices = np.array([[(imshape[1] * 0,imshape[0]),(imshape[1]*0.46, imshape[0]*0.6), (imshape[1]* 0.54, imshape[0]* 0.6), (imshape[1] * 1,imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices=vertices)
    # apply Hough to find lines in the detected edges
    line_img = hough_lines(masked_edges, rho=2, theta=np.pi/180, threshold=15, min_line_len=40, max_line_gap=20)
    # add the detected lines to the original image
    image_with_lines = weighted_img(line_img, initial_img=image)
    plt.figure(i)
    i = i + 1 
    plt.imshow(image_with_lines)
    # save to the test_images_output directory
    mpimg.imsave('test_images_output/' + image_name , image_with_lines)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`


In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [6]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    # read and grayscale the image
    imshape = image.shape  # get the image shape
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # apply Gaussian smoothing
    blur_gray = gaussian_blur(gray, kernel_size=5)
    # apply Canny Edge Detection
    edges = canny(blur_gray, low_threshold=50, high_threshold=150)
    # mask the region we are not intersted in
    vertices = np.array([[(imshape[1] * 0,imshape[0]),(imshape[1]*0.46, imshape[0]*0.6), (imshape[1]* 0.54, imshape[0]* 0.6), (imshape[1] * 1,imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices=vertices)
    # apply Hough to find lines in the detected edges
    line_img = hough_lines(masked_edges, rho=2, theta=np.pi/180, threshold=15, min_line_len=40, max_line_gap=20)
    # add the detected lines to the original image
    result = weighted_img(line_img, initial_img=image)
    
    return result


Let's try the one with the solid white lane on the right first ...

In [7]:
white_output = 'test_videos_output/solidWhiteRight_annotated.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:  12%|█▏        | 15/125 [00:00<00:00, 141.72it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRight_annotated.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight_annotated.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight_annotated.mp4
CPU times: user 2.61 s, sys: 169 ms, total: 2.77 s
Wall time: 1.64 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [9]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

t:  11%|█         | 14/125 [00:00<00:00, 139.75it/s, now=None]

Moviepy - Building video test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeft.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidYellowLeft.mp4
CPU times: user 2.62 s, sys: 149 ms, total: 2.76 s
Wall time: 1.61 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [11]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

t:   4%|▎         | 9/251 [00:00<00:02, 88.58it/s, now=None]

Moviepy - Building video test_videos_output/challenge.mp4.
Moviepy - Writing video test_videos_output/challenge.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/challenge.mp4
CPU times: user 8.65 s, sys: 426 ms, total: 9.08 s
Wall time: 6.79 s


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))